# Semi-automatic morphological analysis of neurons (Step 2: preprocessing Ilastik predictions)
- Goal: Measure neurite volume and arbor widths of a target cell in a confocal image stack. 
- Ilastik pixel classified can be used to seperate the neurite of interest from noise and off-target cells. 
- Thresholding/object segmentation does not seem to be working properly in the version of Ilastik I am using. Even when this worked in the past, exploring parameters in Ilastik is clunky and slow, so this should be done externally (here). 

Preprocessing Ilastik prob maps
- Neurite probability map is thresholded and binarized
- Connected components in the Binary map is labelled
- Ideally: the neurite of interest will be the largest connected component, allowing us to filter out everything else


In [1]:
%matplotlib ipympl
%load_ext autoreload
%autoreload

import matplotlib.pyplot as plt
import mpl_interactions.ipyplot as iplt
import h5py
import os.path
from typing import Tuple, Dict, Union
import numpy as np
import pandas as pd
from skimage import io
import skimage.morphology
from typing import List

from fig_utils import rand_cmap
plt.style.use('mystyle.mplstyle')
#import PIL
# import mpl_interactions.ipyplot as iplt

## Load data
- Ilastik exports a pmap for each label you trained the classifier on. (even when you tell it to export it for only one label) 
- In this case, the first map is the neurite probabilities, the second is the background

In [3]:
def read_h5_multi_channel(fp: str, channels: Union[List, str]=None) -> np.ndarray:
        
    f = h5py.File(fp, "r")
    arr = []
    
    if channels is not None:
        for ch in channels:
            arr.append(np.array(f[ch], dtype=int))
        print(f'Channels included: {channels}')
    else:
        arr = np.array([f[ch] for ch in f.keys()])
        print(f'Channels not specified. All are included: {list(f.keys())}')
    f.close()
    return np.squeeze(np.array(arr)) # singleton dimension removed if len(channels) == 1


In [10]:
def read_h5_pmap(fp: str):
    f = h5py.File(fp, "r")
    
    
    data = np.array(f['pmap'])
    return data
    
p = read_h5_pmap(os.path.expanduser('~/Barnhart/data/mito-seg/ilastik/dsets-combined/211118_mitoseg_neurites-combined.h5'))

In [11]:
p.shape

(116, 364, 364, 2)

In [3]:
data_dir = os.path.expanduser('~/barnhart-lab/data/mito-seg/')
#single_channel = ['tdTomato']
dsets = ['']
im_path = os.path.join(data_dir, f'image-data/{dset}.tif')
#im_path = os.path.expanduser(data_dir + 'h5_files/3.control_axon-039_140620fly1.h5')

if im_path[-3: ] == '.h5':
    im = read_h5_multi_channel(im_path, single_channel)
else:
    im = io.imread(im_path)
mip = np.max(im, axis=0)

In [9]:
pmap_path = os.path.join(data_dir, f"ilastik-data/probability_maps/1.210923_prob.h5")
f = h5py.File(pmap_path, "r")
p1 = np.array(f['exported_data'], dtype='float32')[:, :, :, 0]  # neurite probability map
p2 = np.array(f['exported_data'], dtype='float32')[:, :, :, 1]  # background p map

# p1 = np.array(f['pmap'], dtype='float32')[:, :, :, 0]  # neurite probability map
# p2 = np.array(f['pmap'], dtype='float32')[:, :, :, 1]  # background p map
f.close()

print(len(p1.flatten()))
print((p1.flatten() > 0.95).sum())
print((p1.flatten() == 1.0).sum())
assert(all([d == dd for d, dd in zip(p1.shape, im.shape)]))  # check image and pmap dims match

nz, ny, nx = p1.shape

14680064
382062
327040


## Examine the probability map
- The probability map exported from Ilastik is a matrix with the probability that each voxel belongs to the neurite of interest. 
- Ideally, you want to see a distribution with few intermediary values (voxels where your model felt uncertain). 
- The porportion of voxels with intermediary values (e.g. 0.1 < p <= 0.9) is a good metric to watch when you are wondering if the pixel classifier needs more training data (if it does, this should decrease).  
- The relative proportion of values close to 1 vs 0 depend on how much neurite is in your volume  
- The background probability map is just 1 - p(voxel = neurite)

In [10]:
fig, ax = plt.subplots(1, figsize=[3.8, 2.8])
ax.set_title('Neurite probability distribution')
ax.hist(p1.flatten(), log=True)
ax.set_ylabel('Voxels')
ax.set_xlabel('P(voxel=neurite)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'P(voxel=neurite)')

In [11]:
# def uncertainty_ratio(p: np.array, uc_interval: float=0.90) -> float:
    
#     uc_range = [(1-uc_interval)/2.0] 
#     uncertain_vx = (p1.flatten() )
# uncertainty = (p1.flatten() >= 0.9).sum() + 

In [12]:
# Show that the background pmap + neurite pmap = 1 (are complementary)
complement = p1.flatten() + p2.flatten()
print(np.unique(complement))

[1.]


In [13]:
fig, ax = plt.subplots(1, 2, figsize=[3.8, 2.3], sharex=True, sharey=True)
ax[0].set_title(f"Data - z max proj.")
ax[0].imshow(mip, cmap='hot')

ax[1].set_title(f"Pmap - z max proj.")
ax[1].imshow(np.max(p1, axis=0), cmap='hot')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Preprocess probability map?

In [14]:
footprint = skimage.morphology.cube(3)
p1_morphed = skimage.morphology.closing(p1, footprint)


In [15]:
fig, ax = plt.subplots(1, 3, figsize=[4.2, 2.3], sharex=True, sharey=True)
ax[0].set_title(f"MIP - image")
ax[0].imshow(mip, cmap='hot')

ax[1].set_title(f"Pmap - before")
ax[1].imshow(np.max(p1, axis=0), cmap='hot')

ax[2].set_title(f"Pmap - after closing")
ax[2].imshow(np.max(p1_morphed, axis=0), cmap='hot')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Get binary mask
- The most simple way to turn a probability map into a binary map is to apply a single threshold
- TODO: two thresholds (aka hysterisis thresholding in Ilastik). The first 'strict' threshold, p>=t1, will define the core of your object. Remaining voxels will only be labelled if they pass the second 'lax' threshold, p>=t2, AND are adjacent to voxels that passed the first threshold. t1 > t2. Hysterisis thresholding is needed when your object and non-objects are close together. 


In [16]:
def binarize(im_stack: np.array, min_thresh:float) -> np.array:
    binary_map = np.zeros(im_stack.shape, dtype=int)
    binary_map[im_stack >= min_thresh] = 1
    return binary_map 

z_slicer = lambda z, im_stack: im_stack[z]

In [17]:
fig, ax = plt.subplots(2, 2, sharex=True, sharey=True, figsize=[3, 3])

def z_thresh_bin0(min_thresh: float, z: int) -> np.array:
    return z_slicer(z, binarize(p1, min_thresh))

ax[0, 0].set_title(f"Max intensity projection")
ax[0, 0].imshow(mip, cmap='hot')

ax[0, 1].set_title(f"Bmap-before closing")
z_range = np.arange(0, p1.shape[0], 1)
thresh_range = np.arange(0, 1, 0.1)
control = iplt.imshow(z_thresh_bin0, min_thresh=thresh_range, z=z_range, cmap='binary', ax=ax[0, 1])

# Dilated
footprint = skimage.morphology.ball(3)  # sphere with radius 3 pixels used as structuring element
p1_morphed = skimage.morphology.closing(p1, footprint)

def z_thresh_bin1(min_thresh: float, z: int) -> np.array:
    return z_slicer(z, binarize(p1_morphed, min_thresh))

ax[1, 0].set_title(f"Pmap-after closing")
ax[1, 0].imshow(np.max(p1_morphed, axis=0), cmap='hot')

ax[1, 1].set_title(f"Bmap-after closing")
z_range = np.arange(0, p1.shape[0], 1)
thresh_range = np.arange(0, 1, 0.1)
control = iplt.imshow(z_thresh_bin1, min_thresh=thresh_range, z=z_range, cmap='hot', ax=ax[1, 1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
fig, ax = plt.subplots(1, 2, sharex=True, sharey=True)

def p1_f(min_thresh: float, z: int) -> np.array:
    return z_slicer(z, binarize(p1_morphed, min_thresh))

ax[0].set_title(f"Max intensity z proj (z={im.shape[0]})")
ax[0].imshow(mip, cmap='hot')

ax[1].set_title(f"Binary mask")
z_range = np.linspace(0, p1.shape[0], num=p1.shape[0], dtype=int)
thresh_range = np.linspace(0.0, 1.0, num=20)
control = iplt.imshow(p1_f, min_thresh=thresh_range, z=z_range, cmap='binary', ax=ax[1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
bin_im = binarize(p1_morphed, 0.9)

fig, ax = plt.subplots(1, figsize=[2.2, 2.2])
ax.set_title('Binary mask (p>=0.90)')
ax.imshow(bin_im.max(axis=0), cmap='Greys', aspect='equal')

#Fake test data
# bin_im = np.array([[[1, 1, 0, 0, 0], 
#                     [1, 0, 0, 1, 1], 
#                     [0, 1, 0, 1, 1]],
#                    [[1, 0, 0, 0, 0], 
#                     [1, 0, 0, 0, 0], 
#                     [0, 1, 0, 0, 0]], 
#                    [[1, 0, 0, 0, 0],
#                     [0, 1, 0, 0, 0], 
#                     [0, 0, 0, 0, 0]]])

# TODO: Try different treshold values, though this might need to be tuned for each dataset 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Morphological operations
- Try dilation
- Closing to get rid of the pepper

In [20]:
# bin_im = binarize(p1, 0.95)

# def plot_comparison(original, result, z):
#     fig, (ax0, ax1) = plt.subplots(1, 2, figsize=[3.5, 2.5], sharex=True, sharey=True)
#     ax0.imshow(original[z], cmap=plt.cm.gray)
#     ax0.set_title(f'original z={z}')
#     ax0.axis('off')
    
#     ax1.imshow(result[z], cmap=plt.cm.gray)
#     ax1.set_title(f'transformed z={z}')
#     ax1.axis('off')
    

# footprint = skimage.morphology.ball(3) # shape and size of structuring element
# dilated = skimage.morphology.binary_dilation(bin_im, footprint) #footprint)

# plot_comparison(bin_im, dilated, z=4)

# #bin_im = dilated

# bin_im = binarize(p1, 0.95)
# closed = skimage.morphology.binary_closing(bin_im, footprint)

# plot_comparison(bin_im, closed, z=4)

    

## Label connected components
- [Hoshen-Kopelman algorithm](https://www.ocf.berkeley.edu/~fricke/projects/hoshenkopelman/hoshenkopelman.html) 
- HK is an algorithm that relies on a union-find data structure that recursively populates equivalent sets. 

In [21]:

# # initialize: the root of each index is itself
# r = {n: n for n in np.arange(0, nz*nx*ny, dtype=int)}
# # blank matrix to store root labels
# label_mat = np.zeros(bin_im.shape, dtype=int)
# largest_label = 0 # this number goes up as more root nodes are defined

# def uf_union(a: int, b: int):
#     """
#     Set index a and b as belonging to the same equivalent set
#     (ie they have the same root node)
#     Updates global variable r
#     """
#     global r
    
#     if a < b: # The larger index gets reassigned to the smaller's root
#         r[uf_find(a)] = uf_find(b)
#     else:
#         r[uf_find(b)] = uf_find(a)
    
# def uf_find(a: int) -> int:
#     """
#     Find the root node of a given index, a
#     Updates global variable r
#     """
#     global r
    
#     current = a
#     while r[current] != current:
#         current = r[current]
#     else: # The root node is found when r[current]==current
#         this_root = current
#     # path compression 
#     # i.e. reassign intermediary nodes to the found root    
#     current = a
#     while r[current] != current:
#         next_current = r[current] 
#         r[current] = this_root # the root index found above
#         current = next_current # move on to the parent
#     return this_root

# def flat_ind(z: int, y: int, x: int) -> int:
#     return (nx + ny) * z + nx * y + x

# def is_occupied(z: int, y: int, x: int) -> int:
#     # Returns the value of binary map at a coordinate 
#     if (z >= 0) and (y >= 0) and (x >= 0):
#         return bin_im[z, y, x]
#     else: # coordinate is outside the binary map
#         return 0
    
# def neighborhood(z: int, y: int, x: int) -> Tuple:
#     # Checks the left, up, front neighbors if they are occupied
#     # Returns a tuple of 1s and 0s
#     left_occupied = is_occupied(z, y, x - 1)
#     up_occupied = is_occupied(z, y - 1, x)
#     front_occupied = is_occupied(z - 1, y, x)
#     return front_occupied, up_occupied, left_occupied

# #####################################################################
# unoccupied = 0
# for this_z in range(0, nz):
#     for this_y in range(0, ny):
#         for this_x in range(0, nx):
            
#             if is_occupied(this_z, this_y, this_x) == 0:
#                 unoccupied += 1
#             else:  # bin map at xyz is 1
#                 is_front, is_up, is_left = neighborhood(this_z, this_y, this_x)
                
#                 this_ind = flat_ind(this_z, this_y, this_x)
#                 front_ind = flat_ind(this_z - 1, this_y, this_x)
#                 up_ind = flat_ind(this_z, this_y - 1, this_x)
#                 left_ind = flat_ind(this_z, this_y, this_x - 1)
                
#                 # 0 occupied neighbors #####
#                 if is_front == 0 and is_up == 0 and is_left == 0: 
#                     #print(f"{this_x}, {this_y}, {this_z}")
#                     # Make a new label containing this voxel
#                     largest_label += 1  # label 0 reserved for background
#                     r[this_ind] = largest_label
#                     label_mat[this_z, this_y, this_x] = largest_label
#                     # val in dict doesn't change cause labels[n] == n
#                 # 1 occupied neighbor ###################
#                 elif is_front > 0 and is_up == 0 and is_left == 0: # front
#                     uf_union(this_ind, front_ind)
#                     label_mat[this_z, this_y, this_x] = uf_find(front_ind)
#                 elif is_up > 0 and is_front == 0 and is_left == 0: # up
#                     uf_union(this_ind, up_ind)
#                     label_mat[this_z, this_y, this_x] = uf_find(up_ind)
#                 elif is_left > 0 and is_front == 0 and is_up == 0: # left
#                     uf_union(this_ind, left_ind)
#                     label_mat[this_z, this_y, this_x] = uf_find(left_ind)
#                 # 2 occupied neighbors #########################
#                 elif is_front > 0 and is_up > 0 and is_left == 0: # front & up
#                     uf_union(front_ind, up_ind)
#                     uf_union(this_ind, front_ind)
#                     label_mat[this_z, this_y, this_x] = uf_find(front_ind)
#                 elif is_front > 0 and is_left > 0 and is_up == 0: # front & left
#                     uf_union(front_ind, left_ind)
#                     uf_union(this_ind, front_ind)
#                     label_mat[this_z, this_y, this_x] = uf_find(front_ind)
#                 elif is_up > 0 and is_left > 0 and is_front == 0: # up & left
#                     uf_union(up_ind, left_ind)
#                     uf_union(this_ind, up_ind)
#                     label_mat[this_z, this_y, this_x] = uf_find(up_ind)
#                 # 3 occupied neighbors ###############################
#                 else:
#                     # U(front, up) = U(front, left)?
#                     uf_union(front_ind, up_ind)
#                     uf_union(front_ind, left_ind)
#                     #assert(p == q)
#                     label_mat[this_z, this_y, this_x] = uf_find(front_ind)


In [22]:
# #bin_im = binarize(p1, 0.9)

# bin_im = np.array([[[1, 1, 0, 0, 0], 
#                     [1, 0, 0, 1, 1], 
#                     [0, 1, 0, 1, 1]],
#                    [[1, 0, 0, 0, 0], 
#                     [1, 0, 0, 0, 0], 
#                     [0, 1, 0, 0, 0]], 
#                    [[1, 0, 0, 0, 0],
#                     [0, 1, 0, 0, 0], 
#                     [0, 0, 0, 0, 0]]])
# fig, ax = plt.subplots()
# ax.imshow(bin_im[2])

In [23]:
# fig, ax = plt.subplots(1, 3, figsize=[3.5, 1.5])
# ax[0].imshow(bin_im[0], cmap='gray')
# ax[1].imshow(bin_im[1], cmap='gray')
# ax[2].imshow(bin_im[2], cmap='gray')
thresh = 0.90
fig, ax = plt.subplots(1, figsize=[2.5, 2.5])

label_mat, n_labels = skimage.morphology.label(binarize(p1_morphed, min_thresh=thresh), background=0, return_num=True)

def label_f(z: int) -> np.array:
    return z_slicer(z, label_mat)

ax.set_title(f"Connected components")

z_range = np.arange(0, bin_im.shape[0], dtype=int)

control = iplt.imshow(label_f, z=z_range, cmap=rand_cmap(len(np.unique(label_mat))), autoscale_cmap=False, ax=ax)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Remove small components

In [24]:
labels, n_px = np.unique(label_mat, return_counts=True)

labels_npx = pd.Series(n_px, index=labels).sort_values(ascending=False)
ordered_labels = labels_npx.index

print(f"Proportion background: {labels_npx.iloc[0]/(nx*ny*nz)}")
print(f"Proportion largest component: {labels_npx.iloc[1]/(nx*ny*nz)}")

bg_label =  ordered_labels[0]
neurite_label = ordered_labels[1]

neurite_mat = label_mat == neurite_label
neurite_mat = neurite_mat.astype('uint8')
display(neurite_mat[0])

fig, ax = plt.subplots(1)

def z_neurite_mat(z: int) -> np.array:
    return z_slicer(z, neurite_mat)

ax.set_title(f"Neurite label")

z_range = np.arange(0, neurite_mat.shape[0], dtype=int)

control = iplt.imshow(z_neurite_mat, z=z_range, cmap='binary', autoscale_cmap=False, ax=ax)
    

Proportion background: 0.9667869976588658
Proportion largest component: 0.03283895765032087


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Skeletonize

In [25]:
skel_mat = skimage.morphology.skeletonize(neurite_mat)

print(skel_mat.dtype)
print(neurite_mat.dtype)

fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=[8.5, 3.5])

def z_skel_mat(z: int) -> np.array:
    return z_slicer(z, skel_mat)

ax[0].set_title(f"Image (z-proj)")
ax[1].set_title(f"Binary mask (z-proj)")
ax[2].set_title("Skeleton (z-proj)")

z_range = np.arange(0, neurite_mat.shape[0], dtype=int)
ax[0].imshow(mip, cmap='hot')
ax[1].imshow(neurite_mat.max(axis=0), cmap='binary')
ax[2].imshow(skel_mat.sum(axis=0), cmap='binary')
#control = iplt.imshow(z_skel_mat, z=z_range, cmap='binary', autoscale_cmap=False, ax=ax[1])

uint8
uint8


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
output_file = os.path.join(data_dir, f'{dset}.h5')


f = h5py.File(output_file, 'w')
im_dset = f.create_dataset('im', data=im)
p_dset = f.create_dataset('pmap', data=p1)
n_dset = f.create_dataset('neurite', data=neurite_mat)
s_dset = f.create_dataset('skel', data=skel_mat)
f.close()


In [25]:
type(im[0, 0, 0])

numpy.uint8

In [42]:
# footprint = skimage.morphology.cube(2)
# eroded = skimage.morphology.erosion(neurite_mat)
# skel_mat = skimage.morphology.skeletonize(eroded)

# fig, ax = plt.subplots(1, 2, sharex=True, sharey=True)

# def z_skel_mat(z: int) -> np.array:
#     return z_slicer(z, skel_mat)

# #ax.set_title(f"Neurite label")

# z_range = np.arange(0, neurite_mat.shape[0], dtype=int)
# ax[0].imshow(mip, cmap='hot')
# ax[1].imshow(skel_mat.sum(axis=0), cmap='binary')
# #control = iplt.imshow(z_skel_mat, z=z_range, cmap='binary', autoscale_cmap=False, ax=ax[1])

In [43]:
fig, ax = plt.subplots(1, sharex=True, sharey=True)


ax.imshow(mip, cmap='Greys_r', alpha=0.6)
ax.imshow(skel_mat.sum(axis=0), cmap='hot', alpha=0.6)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Next: erosion to disconnect adjacent arbors before skeletonization 

In [44]:
# fig, ax = plt.subplots(1, 3, figsize=[3.5, 1.5])
# ax[0].imshow(bin_im[0], cmap='gray')
# ax[1].imshow(bin_im[1], cmap='gray')
# ax[2].imshow(bin_im[2], cmap='gray')

fig, ax = plt.subplots(1, figsize=[2.5, 2.5])

footprint = skimage.morphology.ball(3) # shape and size of structuring element
dilated = skimage.morphology.binary_dilation(bin_im, footprint) #footprint)

plot_comparison(bin_im, dilated, z=4)
bin_im = dilated

label_mat, n_labels = skimage.morphology.label(bin_im, background=0, return_num=True)

def label_f(z: int) -> np.array:
    return z_slicer(z, label_mat)

ax.set_title(f"Connected components")

z_range = np.arange(0, bin_im.shape[0], dtype=int)

control = iplt.imshow(label_f, z=z_range, cmap=rand_cmap(len(np.unique(label_mat))), autoscale_cmap=False, ax=ax)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'plot_comparison' is not defined

In [ ]:
# Properties of connected components
fig, ax = plt.subplots(1)

pd.Series(label_mat.flatten()).nunique()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=[16, 7], sharex=True)

ax[0].hist((im/255.0).flatten(), bins=300)
ax[1].hist(label1.flatten(), bins=300)

fig.show()

In [ ]:
im_path = os.path.expanduser('~/barnhart-lab/confocal-HS/210921-nicole-images/HSN_channel4.tif')

im = io.imread(im_path)
mip = np.max(im, axis=0)

In [ ]:
pmap_path = os.path.expanduser("~/barnhart-lab/confocal-HS/210923_HSN_prob.h5")
f = h5py.File(pmap_path, "r")
label1 = np.array(f['exported_data'], dtype='float32')[:, :, :, 0]
label2 = np.array(f['exported_data'], dtype='float32')[:, :, :, 1]

fig, ax = plt.subplots(1, 2, figsize=[16, 7])
ax[0].set_title(f"Maximum intensity projection (z={im.shape[0]})")
ax[0].imshow(mip, cmap='hot')

ax[1].set_title(f"2D probability map (z={im.shape[0]})")
ax[1].imshow(np.max(label1, axis=0), cmap='hot')

In [ ]:
fig, ax = plt.subplots()

im_slicer = lambda z: im[z]


z_range = np.arange(0, im.shape[0])  # z slider
z_slider_control = iplt.imshow(im_slicer, z=z_range, cmap='gray', ax=ax)



In [ ]:
fig, ax = plt.subplots()

l1_slicer = lambda z: label1[z]


z_range = np.arange(0, im.shape[0])  # z slider
z_slider_control = iplt.imshow(l1_slicer, z=z_range, cmap='hot', ax=ax)



In [ ]:
fig, ax = plt.subplots()

l2_slicer = lambda z: label2[z]


z_range = np.arange(0, im.shape[0])  # z slider
z_slider_control = iplt.imshow(l2_slicer, z=z_range, cmap='hot', ax=ax)



In [ ]:
fig, ax = plt.subplots()

im_slicer = lambda z: im[z]


z_range = np.arange(0, im.shape[0])  # z slider
z_slider_control = iplt.imshow(z_slicer, z=z_range, cmap='gray', ax=ax)



In [ ]:
label1.T.shape

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=[16, 7], sharex=True, sharey=True)

def z_slicer(z, stack):
    # return a function that returns z-slices of a stack
    return lambda this_z : stack[this_z] 

im_slicer = lambda z: im[z] 
l1_slicer = lambda z: label1[z] 
l2_slicer = lambda z: label2[z] 

z_range = np.arange(0, im.shape[0])  # z slider
z_slider_control = iplt.imshow(im_slicer, z=z_range, cmap='gray', ax=ax[0])
z_slider_control = iplt.imshow(l1_slicer, z=z_range, cmap='gray', ax=ax[1])
z_slider_control = iplt.imshow(l2_slicer, z=z_range, cmap='gray', ax=ax[2])

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=[12, 8])

def z_slicer(z, stack):
    # return a function that returns z-slices of a stack
    return lambda this_z : stack[this_z] 

#im_slicer = lambda z: im[14, :, :, z] 
l1_slicer = lambda z: label1[14, :, :, z] 
l2_slicer = lambda z: label2[14, :, :, z] 

z_range = np.arange(0, im.shape[0])  # z slider
#z_slider_control = iplt.imshow(im_slicer, z=z_range, cmap='gray', ax=ax[0])
z_slider_control = iplt.imshow(l1_slicer, z=z_range, cmap='gray', ax=ax[1])
z_slider_control = iplt.imshow(l2_slicer, z=z_range, cmap='gray', ax=ax[2])

In [ ]:
iplt.imshow(im[10], cmap='gray')